In [1]:
import pandas as pd
import numpy as np
import random
from tqdm import tqdm

pd.set_option('display.max_columns', 50)

In [2]:
reg_data_m =pd.read_csv('data/MRegularSeasonDetailedResults.csv')
reg_data_m['Gender'] = 1 # 1 for Men
reg_data_m.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,WFGM,WFGA,WFGM3,WFGA3,WFTM,WFTA,WOR,WDR,WAst,WTO,WStl,WBlk,WPF,LFGM,LFGA,LFGM3,LFGA3,LFTM,LFTA,LOR,LDR,LAst,LTO,LStl,LBlk,LPF,Gender
0,2003,10,1104,68,1328,62,N,0,27,58,3,14,11,18,14,24,13,23,7,1,22,22,53,2,10,16,22,10,22,8,18,9,2,20,1
1,2003,10,1272,70,1393,63,N,0,26,62,8,20,10,19,15,28,16,13,4,4,18,24,67,6,24,9,20,20,25,7,12,8,6,16,1
2,2003,11,1266,73,1437,61,N,0,24,58,8,18,17,29,17,26,15,10,5,2,25,22,73,3,26,14,23,31,22,9,12,2,5,23,1
3,2003,11,1296,56,1457,50,N,0,18,38,3,9,17,31,6,19,11,12,14,2,18,18,49,6,22,8,15,17,20,9,19,4,3,23,1
4,2003,11,1400,77,1208,71,N,0,30,61,6,14,11,13,17,22,12,14,4,4,20,24,62,6,16,17,27,21,15,12,10,7,1,14,1


In [3]:
ncaa_data_m = pd.read_csv('data/MNCAATourneyDetailedResults.csv')
ncaa_data_m['Gender'] = 1 # 1 for Men
ncaa_data_m.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,WFGM,WFGA,WFGM3,WFGA3,WFTM,WFTA,WOR,WDR,WAst,WTO,WStl,WBlk,WPF,LFGM,LFGA,LFGM3,LFGA3,LFTM,LFTA,LOR,LDR,LAst,LTO,LStl,LBlk,LPF,Gender
0,2003,134,1421,92,1411,84,N,1,32,69,11,29,17,26,14,30,17,12,5,3,22,29,67,12,31,14,31,17,28,16,15,5,0,22,1
1,2003,136,1112,80,1436,51,N,0,31,66,7,23,11,14,11,36,22,16,10,7,8,20,64,4,16,7,7,8,26,12,17,10,3,15,1
2,2003,136,1113,84,1272,71,N,0,31,59,6,14,16,22,10,27,18,9,7,4,19,25,69,7,28,14,21,20,22,11,12,2,5,18,1
3,2003,136,1141,79,1166,73,N,0,29,53,3,7,18,25,11,20,15,18,13,1,19,27,60,7,17,12,17,14,17,20,21,6,6,21,1
4,2003,136,1143,76,1301,74,N,1,27,64,7,20,15,23,18,20,17,13,8,2,14,25,56,9,21,15,20,10,26,16,14,5,8,19,1


In [4]:
#removing WLoc
print(reg_data_m['WLoc'].value_counts())
print(ncaa_data_m['WLoc'].value_counts())
# removing WLoc because the final data to predicted on will only have it as Neutral so not needed

H    63368
A    33239
N    11027
Name: WLoc, dtype: int64
N    1248
Name: WLoc, dtype: int64


In [5]:
# thinking of removing NumOT
print("Regular Season\n",reg_data_m['NumOT'].value_counts())
print("NCAA\n",ncaa_data_m['NumOT'].value_counts())
# since NumOT is generally 0 and it is 1 for very few samples in the NCAA data, i believe it to not contribute a whole lot

Regular Season
 0    101530
1      5056
2       852
3       157
4        33
5         5
6         1
Name: NumOT, dtype: int64
NCAA
 0    1170
1      65
2      13
Name: NumOT, dtype: int64


In [6]:
reg_data_w =pd.read_csv('data/WRegularSeasonDetailedResults.csv')
reg_data_w['Gender'] = 0 # 0 for Women
reg_data_w.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,WFGM,WFGA,WFGM3,WFGA3,WFTM,WFTA,WOR,WDR,WAst,WTO,WStl,WBlk,WPF,LFGM,LFGA,LFGM3,LFGA3,LFTM,LFTA,LOR,LDR,LAst,LTO,LStl,LBlk,LPF,Gender
0,2010,11,3103,63,3237,49,H,0,23,54,5,9,12,19,10,26,14,18,7,0,15,20,54,3,13,6,10,11,27,11,23,7,6,19,0
1,2010,11,3104,73,3399,68,N,0,26,62,5,12,16,28,16,31,15,20,5,2,25,25,63,4,21,14,27,14,26,7,20,4,2,27,0
2,2010,11,3110,71,3224,59,A,0,29,62,6,15,7,12,14,23,18,13,6,2,17,19,58,2,14,19,23,17,23,8,15,6,0,15,0
3,2010,11,3111,63,3267,58,A,0,27,52,4,11,5,9,6,40,14,27,5,10,18,18,74,6,26,16,25,22,22,15,11,14,5,14,0
4,2010,11,3119,74,3447,70,H,1,30,74,7,20,7,11,14,33,18,11,5,3,18,25,74,9,17,11,21,21,32,12,14,4,2,14,0


In [7]:
ncaa_data_w = pd.read_csv('data/WNCAATourneyDetailedResults.csv')
ncaa_data_w['Gender'] = 0 # 0 for Women
ncaa_data_w.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,WFGM,WFGA,WFGM3,WFGA3,WFTM,WFTA,WOR,WDR,WAst,WTO,WStl,WBlk,WPF,LFGM,LFGA,LFGM3,LFGA3,LFTM,LFTA,LOR,LDR,LAst,LTO,LStl,LBlk,LPF,Gender
0,2010,138,3124,69,3201,55,N,0,28,57,1,5,12,19,13,24,22,12,6,2,12,21,61,10,34,3,5,17,19,12,18,4,1,18,0
1,2010,138,3173,67,3395,66,N,0,23,59,9,26,12,19,13,34,13,16,3,10,14,22,73,8,27,14,15,18,26,8,8,8,6,22,0
2,2010,138,3181,72,3214,37,H,0,26,57,4,13,16,22,13,34,15,11,10,7,11,15,56,4,15,3,8,10,21,4,16,6,4,20,0
3,2010,138,3199,75,3256,61,H,0,25,63,3,15,22,26,20,27,13,17,8,3,21,21,62,2,20,17,22,16,21,13,16,5,4,24,0
4,2010,138,3207,62,3265,42,N,0,24,68,8,25,6,8,20,29,16,8,5,5,18,13,60,5,26,11,17,16,22,9,10,3,4,12,0


In [8]:
#removing WLoc
print(reg_data_w['WLoc'].value_counts())
print(ncaa_data_w['WLoc'].value_counts())
# removing WLoc because the final data to predicted on will only have it as Neutral so not needed

H    39117
A    25410
N     6256
Name: WLoc, dtype: int64
N    492
H    228
A     40
Name: WLoc, dtype: int64


In [9]:
# thinking of removing NumOT
print("Regular Season\n",reg_data_w['NumOT'].value_counts())
print("NCAA\n",ncaa_data_w['NumOT'].value_counts())
# since NumOT is generally 0 and it is 1 for very few samples in the NCAA data, i believe it to not contribute a whole lot

Regular Season
 0    67642
1     2667
2      407
3       55
4       11
5        1
Name: NumOT, dtype: int64
NCAA
 0    734
1     23
2      3
Name: NumOT, dtype: int64


In [10]:
data = pd.concat([reg_data_m,ncaa_data_m,reg_data_w,ncaa_data_w])
print(data.shape)
print(data['Season'].value_counts())

(180425, 35)
2023    10976
2019    10833
2018    10744
2017    10735
2014    10718
2016    10708
2015    10698
2013    10593
2022    10539
2020    10499
2011    10460
2012    10433
2010    10427
2021     7540
2009     5313
2008     5227
2007     5107
2006     4821
2005     4739
2003     4680
2004     4635
Name: Season, dtype: int64


In [11]:
drop_cols = ['WLoc','NumOT']
reg_data = data.drop(columns = drop_cols)
cols = reg_data.columns.values
print(cols)
reg_data.head()

['Season' 'DayNum' 'WTeamID' 'WScore' 'LTeamID' 'LScore' 'WFGM' 'WFGA'
 'WFGM3' 'WFGA3' 'WFTM' 'WFTA' 'WOR' 'WDR' 'WAst' 'WTO' 'WStl' 'WBlk'
 'WPF' 'LFGM' 'LFGA' 'LFGM3' 'LFGA3' 'LFTM' 'LFTA' 'LOR' 'LDR' 'LAst'
 'LTO' 'LStl' 'LBlk' 'LPF' 'Gender']


,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WFGM,WFGA,WFGM3,WFGA3,WFTM,WFTA,WOR,WDR,WAst,WTO,WStl,WBlk,WPF,LFGM,LFGA,LFGM3,LFGA3,LFTM,LFTA,LOR,LDR,LAst,LTO,LStl,LBlk,LPF,Gender
0,2003,10,1104,68,1328,62,27,58,3,14,11,18,14,24,13,23,7,1,22,22,53,2,10,16,22,10,22,8,18,9,2,20,1
1,2003,10,1272,70,1393,63,26,62,8,20,10,19,15,28,16,13,4,4,18,24,67,6,24,9,20,20,25,7,12,8,6,16,1
2,2003,11,1266,73,1437,61,24,58,8,18,17,29,17,26,15,10,5,2,25,22,73,3,26,14,23,31,22,9,12,2,5,23,1
3,2003,11,1296,56,1457,50,18,38,3,9,17,31,6,19,11,12,14,2,18,18,49,6,22,8,15,17,20,9,19,4,3,23,1
4,2003,11,1400,77,1208,71,30,61,6,14,11,13,17,22,12,14,4,4,20,24,62,6,16,17,27,21,15,12,10,7,1,14,1


In [12]:
# converting it to team A and team B from WTeam and LTeam
list_reg_data = []

for row in tqdm(reg_data.itertuples()):
    new_reg_data = dict()
    coin = random.random()
    new_reg_data['Season'] = row.Season
    new_reg_data['DayNum'] = row.DayNum

    if coin>0.5:
        new_reg_data['A'] = row.WTeamID
        new_reg_data['B'] = row.LTeamID
        new_reg_data['Gender'] = row.Gender
        try:
            new_reg_data['FGP2_A'] = ((row.WFGM - row.WFGM3)/ (row.WFGA - row.WFGA3)) * 100
        except:
            new_reg_data['FGP2_A'] = 0
        try:
            new_reg_data['FGP3_A'] = (row.WFGM3 / row.WFGA3) * 100
        except:
            new_reg_data['FGP3_A'] = 0
        try:
            new_reg_data['FTP_A'] = (row.WFTM / row.WFTA) * 100
        except:
            new_reg_data['FTP_A'] = 0
        try:
            new_reg_data['AST_TO_A'] = row.WAst/row.WTO
        except:
            new_reg_data['AST_TO_A'] = row.WAst/(row.WTO+1)
        try:
            new_reg_data['STBL_PF_A'] = (row.WStl + row.WBlk)/row.WPF # steal + block / personal foul
        except:
            new_reg_data['STBL_PF_A'] = (row.WStl + row.WBlk)/(row.WPF+1)
        new_reg_data['PM_A'] = row.WScore - row.LScore # plus minus for team A
        new_reg_data['POSS_A'] = row.WFGA - row.WOR + row.WTO + (0.475*row.WFTA) # https://thepowerrank.com/cbb-analytics/ possession
        new_reg_data['REB_DIFF_A'] = (row.WOR/(row.WFGA-row.WFGM)) - (row.WDR/(row.LFGA-row.LFGM))
        try:
            new_reg_data['FGP2_B'] = ((row.LFGM - row.LFGM3)/ (row.LFGA - row.LFGA3)) * 100
        except:
            new_reg_data['FGP2_B'] = 0
        try:
            new_reg_data['FGP3_B'] = (row.LFGM3 / row.LFGA3) * 100
        except:
            new_reg_data['FGP3_B'] = 0
        try:
            new_reg_data['FTP_B'] = (row.LFTM / row.LFTA) * 100
        except:
            new_reg_data['FTP_B'] = 0
        try:
            new_reg_data['AST_TO_B'] = row.LAst/row.LTO
        except:
            new_reg_data['AST_TO_B'] = row.LAst/(row.LTO+1)
        try:
            new_reg_data['STBL_PF_B'] = (row.LStl + row.LBlk)/row.LPF
        except:
            new_reg_data['STBL_PF_B'] = (row.LStl + row.LBlk)/(row.LPF+1)
        new_reg_data['PM_B'] = row.LScore - row.WScore # plus minus for team B
        new_reg_data['POSS_B'] = row.LFGA - row.LOR + row.LTO + (0.475*row.LFTA)
        new_reg_data['REB_DIFF_B'] = (row.LOR/(row.LFGA-row.LFGM)) - (row.LDR/(row.WFGA-row.WFGM))
        new_reg_data['Win'] = 1
    else:
        new_reg_data['A'] = row.LTeamID
        new_reg_data['B'] = row.WTeamID
        new_reg_data['Gender'] = row.Gender
        try:
            new_reg_data['FGP2_A'] = ((row.LFGM - row.LFGM3)/ (row.LFGA - row.LFGA3)) * 100
        except:
            new_reg_data['FGP2_A'] = 0
        try:
            new_reg_data['FGP3_A'] = (row.LFGM3 / row.LFGA3) * 100
        except:
            new_reg_data['FGP3_A'] = 0
        try:
            new_reg_data['FTP_A'] = (row.LFTM / row.LFTA) * 100
        except:
            new_reg_data['FTP_A'] = 0
        try:
            new_reg_data['AST_TO_A'] = row.LAst/row.LTO
        except:
            new_reg_data['AST_TO_A'] = row.LAst/(row.LTO+1)
        try:
            new_reg_data['STBL_PF_A'] = (row.LStl + row.LBlk)/row.LPF
        except:
            new_reg_data['STBL_PF_A'] = (row.LStl + row.LBlk)/(row.LPF+1)
        new_reg_data['PM_A'] = row.LScore - row.WScore # plus minus for team A
        new_reg_data['POSS_A'] = row.LFGA - row.LOR + row.LTO + (0.475*row.LFTA)
        new_reg_data['REB_DIFF_A'] = (row.LOR/(row.LFGA-row.LFGM)) - (row.LDR/(row.WFGA-row.WFGM))
        try:
            new_reg_data['FGP2_B'] = ((row.WFGM - row.WFGM3)/ (row.WFGA - row.WFGA3)) * 100
        except:
            new_reg_data['FGP2_B'] = 0
        try:
            new_reg_data['FGP3_B'] = (row.WFGM3 / row.WFGA3) * 100
        except:
            new_reg_data['FGP3_B'] = 0
        try:
            new_reg_data['FTP_B'] = (row.WFTM / row.WFTA) * 100
        except:
            new_reg_data['FTP_B'] = 0
        try:
            new_reg_data['AST_TO_B'] = row.WAst/row.WTO
        except:
            new_reg_data['AST_TO_B'] = row.WAst/(row.WTO+1)
        try:
            new_reg_data['STBL_PF_B'] = (row.WStl + row.WBlk)/row.WPF
        except:
            new_reg_data['STBL_PF_B'] = (row.WStl + row.WBlk)/(row.WPF+1)
        new_reg_data['PM_B'] = row.WScore - row.LScore # plus minus for team B
        new_reg_data['POSS_B'] = row.WFGA - row.WOR + row.WTO + (0.475*row.WFTA)
        new_reg_data['REB_DIFF_B'] = (row.WOR/(row.WFGA-row.WFGM)) - (row.WDR/(row.LFGA-row.LFGM))
        new_reg_data['Win'] = 0

    list_reg_data.append(new_reg_data)

new_reg_data = pd.DataFrame(list_reg_data)
new_reg_data


180425it [00:01, 174022.56it/s]


,Season,DayNum,A,B,Gender,FGP2_A,FGP3_A,FTP_A,AST_TO_A,STBL_PF_A,PM_A,POSS_A,REB_DIFF_A,FGP2_B,FGP3_B,FTP_B,AST_TO_B,STBL_PF_B,PM_B,POSS_B,REB_DIFF_B,Win
0,2003,10,1328,1104,1,46.511628,20.000000,72.727273,0.444444,0.550000,-6,71.450,-0.387097,54.545455,21.428571,61.111111,0.565217,0.363636,6,75.550,-0.322581,0
1,2003,10,1393,1272,1,41.860465,25.000000,45.000000,0.583333,0.875000,-7,68.500,-0.229328,42.857143,40.000000,52.631579,1.230769,0.444444,7,69.025,-0.234496,0
2,2003,11,1266,1437,1,40.000000,44.444444,58.620690,1.500000,0.280000,12,64.775,-0.009804,40.425532,11.538462,60.869565,0.750000,0.304348,-12,64.925,-0.039216,1
3,2003,11,1296,1457,1,51.724138,33.333333,54.838710,0.916667,0.888889,6,58.725,-0.312903,44.444444,27.272727,53.333333,0.473684,0.304348,-6,58.125,-0.451613,1
4,2003,11,1208,1400,1,39.130435,37.500000,62.962963,1.200000,0.571429,-6,63.825,0.068761,51.063830,42.857143,84.615385,0.857143,0.400000,6,64.175,-0.030560,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
180420,2022,147,3301,3163,0,58.139535,30.434783,84.210526,1.538462,0.687500,-4,82.025,-0.573529,57.142857,23.809524,60.000000,1.428571,0.437500,4,81.500,-0.376471,0
180421,2022,147,3257,3276,0,46.511628,33.333333,77.777778,1.090909,1.117647,12,67.275,-0.484848,40.625000,21.428571,75.000000,0.428571,0.666667,-12,66.500,-0.393939,1
180422,2022,151,3163,3390,0,37.209302,35.714286,80.000000,0.736842,0.437500,5,73.500,-0.364341,44.186047,17.391304,61.538462,0.909091,0.875000,-5,72.175,-0.383075,1
180423,2022,151,3257,3376,0,47.272727,12.500000,57.142857,0.333333,0.882353,-13,70.325,-0.294444,52.500000,35.294118,70.588235,1.357143,1.363636,13,71.075,-0.400000,0


In [13]:
new_reg_data['Gender'].value_counts()

1    108882
0     71543
Name: Gender, dtype: int64

In [14]:
new_reg_data.iloc[:,4:].describe()

,Gender,FGP2_A,FGP3_A,FTP_A,AST_TO_A,STBL_PF_A,PM_A,POSS_A,REB_DIFF_A,FGP2_B,FGP3_B,FTP_B,AST_TO_B,STBL_PF_B,PM_B,POSS_B,REB_DIFF_B,Win
count,180425.000000,180425.000000,180425.000000,180425.000000,180425.000000,180425.000000,180425.000000,180425.000000,180425.000000,180425.000000,180425.000000,180425.000000,180425.000000,180425.000000,180425.000000,180425.000000,180425.000000,180425.000000
mean,0.603475,46.828191,33.050352,69.430580,1.028493,0.779486,0.075682,69.421706,-0.401200,46.829817,33.014509,69.399299,1.023194,0.776112,-0.075682,69.415107,-0.400709,0.501605
std,0.489177,9.822343,12.041006,13.083254,0.611191,1.318008,16.345492,6.794276,0.158387,9.819670,12.016977,13.116635,0.604491,1.314305,16.345492,6.789407,0.157853,0.499999
min,0.000000,4.166667,0.000000,0.000000,0.000000,0.000000,-94.000000,41.425000,-1.481707,7.317073,0.000000,0.000000,0.000000,0.000000,-108.000000,41.800000,-1.337121,0.000000
25%,0.000000,40.000000,25.000000,61.538462,0.636364,0.407407,-11.000000,64.800000,-0.506667,40.000000,25.000000,61.538462,0.631579,0.400000,-11.000000,64.800000,-0.505714,0.000000
50%,1.000000,46.666667,33.333333,70.000000,0.909091,0.571429,1.000000,69.025000,-0.400000,46.666667,33.333333,70.000000,0.900000,0.565217,-1.000000,69.025000,-0.399614,1.000000
75%,1.000000,53.333333,40.909091,78.260870,1.272727,0.789474,11.000000,73.550000,-0.294118,53.333333,40.909091,78.260870,1.266667,0.785714,11.000000,73.550000,-0.293569,1.000000
max,1.000000,96.428571,100.000000,100.000000,25.000000,25.000000,108.000000,128.350000,0.512488,94.444444,100.000000,100.000000,29.000000,26.000000,94.000000,126.850000,0.484615,1.000000


In [15]:
new_reg_data['Win'].value_counts()

1    90502
0    89923
Name: Win, dtype: int64

In [16]:
new_reg_data.corr()

,Season,DayNum,A,B,Gender,FGP2_A,FGP3_A,FTP_A,AST_TO_A,STBL_PF_A,PM_A,POSS_A,REB_DIFF_A,FGP2_B,FGP3_B,FTP_B,AST_TO_B,STBL_PF_B,PM_B,POSS_B,REB_DIFF_B,Win
Season,1.000000,-0.020767,0.289158,0.288987,-0.289621,-0.003859,-0.057092,0.057188,0.049574,0.149525,-0.002379,0.130419,-0.151304,-0.006583,-0.049237,0.054009,0.048344,0.149217,0.002379,0.130362,-0.150700,-0.000327
DayNum,-0.020767,1.000000,-0.011442,-0.011064,0.011499,0.012208,0.023414,0.064405,0.064079,-0.041705,-0.003849,-0.151568,-0.051155,0.018706,0.020237,0.062907,0.069845,-0.042024,0.003849,-0.151393,-0.050762,-0.002958
A,0.289158,-0.011442,1.000000,0.988725,-0.994317,-0.255973,-0.130750,-0.006401,-0.151176,-0.014535,0.000939,0.186175,0.171706,-0.258344,-0.128224,-0.007395,-0.153547,-0.014075,-0.000939,0.185989,0.171780,0.001397
B,0.288987,-0.011064,0.988725,1.000000,-0.994357,-0.257384,-0.131188,-0.006714,-0.152385,-0.014289,-0.001992,0.186446,0.170986,-0.256190,-0.128253,-0.006773,-0.152584,-0.013459,0.001992,0.186205,0.172825,-0.001451
Gender,-0.289621,0.011499,-0.994317,-0.994357,1.000000,0.258277,0.131800,0.007412,0.153831,0.014726,0.000588,-0.187888,-0.173332,0.258506,0.129195,0.007988,0.155214,0.014180,-0.000588,-0.187655,-0.174307,-0.000061
FGP2_A,-0.003859,0.012208,-0.255973,-0.257384,0.258277,1.000000,0.108877,0.060228,0.302544,0.048377,0.472162,0.013312,-0.039472,-0.015522,0.007412,0.010698,-0.018228,-0.046692,-0.472162,0.019611,-0.170742,0.377514
FGP3_A,-0.057092,0.023414,-0.130750,-0.131188,0.131800,0.108877,1.000000,0.059046,0.266871,0.000330,0.354875,-0.041908,-0.011259,0.001981,0.009816,0.011404,0.031234,-0.023160,-0.354875,-0.018750,-0.082108,0.302349
FTP_A,0.057188,0.064405,-0.006401,-0.006714,0.007412,0.060228,0.059046,1.000000,0.058698,0.005756,0.157086,0.010623,-0.069753,0.005960,0.012985,0.038729,0.016703,-0.013392,-0.157086,0.008672,0.110395,0.137235
AST_TO_A,0.049574,0.064079,-0.151176,-0.152385,0.153831,0.302544,0.266871,0.058698,1.000000,0.050052,0.421149,-0.112142,-0.078258,-0.018220,0.031191,0.022349,0.046649,-0.075322,-0.421149,-0.102275,-0.100070,0.318699
STBL_PF_A,0.149525,-0.041705,-0.014535,-0.014289,0.014726,0.048377,0.000330,0.005756,0.050052,1.000000,0.102300,0.071070,0.007559,-0.045124,-0.021335,-0.012080,-0.074384,0.813310,-0.102300,0.075687,-0.027532,0.089790


### Few heavy correlation
- FGP_A - FGP3_A (0.53)
- FGP_A - AST_A (0.55)
- FGP_A - DR_B (negative)
- FGP3_A - AST_A
- TO_A - STL_B (quite high, 0.71)
- STL_A - TO_B (quite high, 0.71)
same for B

In [17]:
def get_avg_team(data, team_id, dn):
    data = data[(data['A'] == team_id)|(data['B'] == team_id)]
    data = data[data['DayNum']<dn]
    final_data = {'FGP2_scored':[],'FGP3_scored':[],'FTP_scored':[],
                  'AST_TO_scored':[],'STBL_PF_scored':[],
                  'POSS_scored' : [], 'REB_DIFF_scored':[],
                  'FGP2_allowed':[],'FGP3_allowed':[],'FTP_allowed':[],
                  'AST_TO_allowed':[],'STBL_PF_allowed':[],
                  'POSS_allowed':[], 'REB_DIFF_allowed':[],'PM':[]}
    for row in data.itertuples():
        if row.A == team_id:
            final_data['FGP2_scored'].append(row.FGP2_A)
            final_data['FGP3_scored'].append(row.FGP3_A)
            final_data['FTP_scored'].append(row.FTP_A)
            final_data['AST_TO_scored'].append(row.AST_TO_A)
            final_data['STBL_PF_scored'].append(row.STBL_PF_A)
            final_data['POSS_scored'].append(row.POSS_A)
            final_data['REB_DIFF_scored'].append(row.REB_DIFF_A)
            final_data['FGP2_allowed'].append(row.FGP2_B)
            final_data['FGP3_allowed'].append(row.FGP3_B)
            final_data['FTP_allowed'].append(row.FTP_B)
            final_data['AST_TO_allowed'].append(row.AST_TO_B)
            final_data['STBL_PF_allowed'].append(row.STBL_PF_B)
            final_data['POSS_allowed'].append(row.POSS_B)
            final_data['REB_DIFF_allowed'].append(row.REB_DIFF_B)
            final_data['PM'].append(row.PM_A)
        else:
            final_data['FGP2_scored'].append(row.FGP2_B)
            final_data['FGP3_scored'].append(row.FGP3_B)
            final_data['FTP_scored'].append(row.FTP_B)
            final_data['AST_TO_scored'].append(row.AST_TO_B)
            final_data['STBL_PF_scored'].append(row.STBL_PF_B)
            final_data['POSS_scored'].append(row.POSS_B)
            final_data['REB_DIFF_scored'].append(row.REB_DIFF_B)
            final_data['FGP2_allowed'].append(row.FGP2_A)
            final_data['FGP3_allowed'].append(row.FGP3_A)
            final_data['FTP_allowed'].append(row.FTP_A)
            final_data['AST_TO_allowed'].append(row.AST_TO_A)
            final_data['STBL_PF_allowed'].append(row.STBL_PF_A)
            final_data['POSS_allowed'].append(row.POSS_A)
            final_data['REB_DIFF_allowed'].append(row.REB_DIFF_A)
            final_data['PM'].append(row.PM_B)

    try:
        return [sum(v)/len(v) for v in final_data.values()]
    except:
        return [np.nan]*len(final_data)


def get_avg_yearly(year):
    year_data = new_reg_data[new_reg_data['Season'] == year]
    list_data = []

    for row in tqdm(year_data.itertuples()):
        *stats_A, = get_avg_team(year_data, row.A, row.DayNum)
        *stats_B, = get_avg_team(year_data, row.B, row.DayNum)
        list_data.append([row.Season, row.DayNum, row.A, row.B, row.Gender, *stats_A, *stats_B, row.Win])
    
    data = pd.DataFrame(list_data, columns = ['Season','DayNum','A','B','Gender','FGP2_SCORED_A','FGP3_SCORED_A','FTP_SCORED_A',
                                               'AST_TO_SCORED_A','STBL_PF_SCORED_A','POSS_SCORED_A','REB_DIFF_SCORED_A',
                                               'FGP2_ALLOWED_A', 'FGP3_ALLOWED_A','FTP_ALLOWED_A','AST_TO_ALLOWED_A',
                                               'STBL_PF_ALLOWED_A','POSS_ALLOWED_A','REB_DIFF_ALLOWED_A','PM_A','FGP2_SCORED_B',
                                               'FGP3_SCORED_B','FTP_SCORED_B','AST_TO_SCORED_B','STBL_PF_SCORED_B',
                                               'POSS_SCORED_B','REB_DIFF_SCORED_B','FGP2_ALLOWED_B', 'FGP3_ALLOWED_B','FTP_ALLOWED_B',
                                               'AST_TO_ALLOWED_B', 'STBL_PF_ALLOWED_B','POSS_ALLOWED_B','REB_DIFF_ALLOWED_B','PM_B','Win']).dropna().reset_index(drop=True)

    return data

# to get an understanding of margin difference between the two teams
# Offense_A, Defense_B
# Offense_B, Defense_A
def margin_diff(data):
    final_data = {'Season':[],'DayNum':[],'A':[],'B':[],'Gender':[],'PM_A':[],'PM_B':[],'margin_FGP2_A':[],'margin_FGP3_A':[],'margin_FTP_A':[],'margin_AST_TO_A':[],
                  'margin_STBL_PF_A':[],'margin_REB_DIFF_A':[], 'margin_POSS_A':[],'margin_FGP2_B':[],
                  'margin_FGP3_B':[],'margin_FTP_B':[],'margin_AST_TO_B':[],'margin_STBL_PF_B':[],
                  'margin_REB_DIFF_B':[],'margin_POSS_B':[],'Win':[]}
        
    for row in data.itertuples():
        final_data['Season'].append(row.Season)
        final_data['DayNum'].append(row.DayNum)
        final_data['A'].append(row.A)
        final_data['B'].append(row.B)
        final_data['Gender'].append(row.Gender)
        final_data['PM_A'].append(row.PM_A)
        final_data['PM_B'].append(row.PM_B)
        final_data['margin_FGP2_A'].append(row.FGP2_SCORED_A - row.FGP2_ALLOWED_B)
        final_data['margin_FGP3_A'].append(row.FGP3_SCORED_A - row.FGP3_ALLOWED_B)
        final_data['margin_FTP_A'].append(row.FTP_SCORED_A - row.FTP_ALLOWED_B)
        final_data['margin_AST_TO_A'].append(row.AST_TO_SCORED_A - row.AST_TO_ALLOWED_B)
        final_data['margin_STBL_PF_A'].append(row.STBL_PF_SCORED_A - row.STBL_PF_ALLOWED_B)
        final_data['margin_REB_DIFF_A'].append(row.REB_DIFF_SCORED_A - row.REB_DIFF_ALLOWED_B)
        final_data['margin_POSS_A'].append(row.POSS_SCORED_A - row.POSS_ALLOWED_B)
        final_data['margin_FGP2_B'].append(row.FGP2_SCORED_B - row.FGP2_ALLOWED_A)
        final_data['margin_FGP3_B'].append(row.FGP3_SCORED_B - row.FGP3_ALLOWED_A)
        final_data['margin_FTP_B'].append(row.FTP_SCORED_B - row.FTP_ALLOWED_A)
        final_data['margin_AST_TO_B'].append(row.AST_TO_SCORED_B - row.AST_TO_ALLOWED_A)
        final_data['margin_STBL_PF_B'].append(row.STBL_PF_SCORED_B - row.STBL_PF_ALLOWED_A)
        final_data['margin_REB_DIFF_B'].append(row.REB_DIFF_SCORED_B - row.REB_DIFF_ALLOWED_A)
        final_data['margin_POSS_B'].append(row.POSS_SCORED_B - row.POSS_ALLOWED_A)
        final_data['Win'].append(row.Win)

    final_data = pd.DataFrame(final_data)
    return final_data

def get_data_reg():
    year = [2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023]
    data = []
    for y in tqdm(year):
        data.append(get_avg_yearly(y))

    final_data = pd.concat(data)
    final_data = margin_diff(final_data)
    return final_data


In [18]:
data_03 = get_avg_yearly(2003)


4680it [00:12, 389.51it/s]


In [19]:
data_03.head()

,Season,DayNum,A,B,Gender,FGP2_SCORED_A,FGP3_SCORED_A,FTP_SCORED_A,AST_TO_SCORED_A,STBL_PF_SCORED_A,POSS_SCORED_A,REB_DIFF_SCORED_A,FGP2_ALLOWED_A,FGP3_ALLOWED_A,FTP_ALLOWED_A,AST_TO_ALLOWED_A,STBL_PF_ALLOWED_A,POSS_ALLOWED_A,REB_DIFF_ALLOWED_A,PM_A,FGP2_SCORED_B,FGP3_SCORED_B,FTP_SCORED_B,AST_TO_SCORED_B,STBL_PF_SCORED_B,POSS_SCORED_B,REB_DIFF_SCORED_B,FGP2_ALLOWED_B,FGP3_ALLOWED_B,FTP_ALLOWED_B,AST_TO_ALLOWED_B,STBL_PF_ALLOWED_B,POSS_ALLOWED_B,REB_DIFF_ALLOWED_B,PM_B,Win
0,2003,12,1186,1457,1,48.571429,27.272727,70.588235,0.421053,0.280000,67.075,-0.478908,44.444444,50.000000,85.185185,1.333333,0.666667,66.825,-0.535980,-26.0,44.444444,27.272727,53.333333,0.473684,0.304348,58.125,-0.451613,51.724138,33.333333,54.838710,0.916667,0.888889,58.725,-0.312903,-6.0,1
1,2003,12,1458,1296,1,44.444444,50.000000,85.185185,1.333333,0.666667,66.825,-0.535980,48.571429,27.272727,70.588235,0.421053,0.280000,67.075,-0.478908,26.0,51.724138,33.333333,54.838710,0.916667,0.888889,58.725,-0.312903,44.444444,27.272727,53.333333,0.473684,0.304348,58.125,-0.451613,6.0,1
2,2003,14,1135,1125,1,48.648649,31.578947,64.705882,0.944444,0.923077,68.075,-0.087500,44.680851,26.315789,69.230769,0.882353,0.882353,68.175,-0.193750,-1.0,25.000000,33.333333,50.000000,0.705882,0.588235,70.800,-0.534362,45.454545,43.478261,69.565217,1.384615,2.538462,71.925,-0.502805,-28.0,0
3,2003,14,1236,1156,1,57.692308,26.666667,71.428571,0.366667,0.500000,75.300,-0.247159,44.680851,25.000000,82.051282,0.823529,0.480000,77.525,-0.411932,-18.0,52.941176,33.333333,44.444444,0.520000,0.555556,76.825,-0.402381,48.936170,45.454545,55.555556,0.529412,0.478261,74.550,-0.485714,-5.0,0
4,2003,14,1161,1194,1,44.680851,25.000000,82.051282,0.823529,0.480000,77.525,-0.411932,57.692308,26.666667,71.428571,0.366667,0.500000,75.300,-0.247159,18.0,48.936170,45.454545,55.555556,0.529412,0.478261,74.550,-0.485714,52.941176,33.333333,44.444444,0.520000,0.555556,76.825,-0.402381,5.0,1


In [20]:
data_03_margin = margin_diff(data_03)

In [21]:
data_03_margin

,Season,DayNum,A,B,Gender,PM_A,PM_B,margin_FGP2_A,margin_FGP3_A,margin_FTP_A,margin_AST_TO_A,margin_STBL_PF_A,margin_REB_DIFF_A,margin_POSS_A,margin_FGP2_B,margin_FGP3_B,margin_FTP_B,margin_AST_TO_B,margin_STBL_PF_B,margin_REB_DIFF_B,margin_POSS_B,Win
0,2003,12,1186,1457,1,-26.000000,-6.000000,-3.152709,-6.060606,15.749526,-0.495614,-0.608889,-0.166005,8.350000,0.000000,-22.727273,-31.851852,-0.859649,-0.362319,0.084367,-8.700000,1
1,2003,12,1458,1296,1,26.000000,6.000000,0.000000,22.727273,31.851852,0.859649,0.362319,-0.084367,8.700000,3.152709,6.060606,-15.749526,0.495614,0.608889,0.166005,-8.350000,1
2,2003,14,1135,1125,1,-1.000000,-28.000000,3.194103,-11.899314,-4.859335,-0.440171,-1.615385,0.415305,-3.850000,-19.680851,7.017544,-19.230769,-0.176471,-0.294118,-0.340612,2.625000,0
3,2003,14,1236,1156,1,-18.000000,-5.000000,8.756137,-18.787879,15.873016,-0.162745,0.021739,0.238555,0.750000,8.260325,8.333333,-37.606838,-0.303529,0.075556,0.009551,-0.700000,0
4,2003,14,1161,1194,1,18.000000,5.000000,-8.260325,-8.333333,37.606838,0.303529,-0.075556,-0.009551,0.700000,-8.756137,18.787879,-15.873016,0.162745,-0.021739,-0.238555,-0.750000,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4472,2003,146,1393,1328,1,10.000000,11.121212,7.684522,2.151693,3.205801,0.401816,0.544782,0.136700,7.471236,3.491727,9.549893,3.188075,0.125275,0.075790,-0.103482,-7.814157,1
4473,2003,146,1400,1277,1,10.322581,6.852941,4.218556,2.519487,1.009623,0.288299,0.022799,0.141285,6.022723,4.743501,2.547989,8.203791,0.069595,-0.015639,-0.159746,-5.349407,1
4474,2003,152,1242,1266,1,14.441176,10.312500,7.885769,2.281373,-4.211293,0.106029,0.510847,-0.013997,6.556939,9.387248,4.917044,8.922371,0.565818,-0.050506,0.133548,-5.333869,1
4475,2003,152,1400,1393,1,10.281250,10.181818,4.522181,5.196445,5.675823,0.052845,-0.018130,0.012254,-1.452699,8.414924,-1.136568,4.869187,0.288810,0.491693,0.015461,1.951042,0


In [22]:
data_03_margin.corr()

,Season,DayNum,A,B,Gender,PM_A,PM_B,margin_FGP2_A,margin_FGP3_A,margin_FTP_A,margin_AST_TO_A,margin_STBL_PF_A,margin_REB_DIFF_A,margin_POSS_A,margin_FGP2_B,margin_FGP3_B,margin_FTP_B,margin_AST_TO_B,margin_STBL_PF_B,margin_REB_DIFF_B,margin_POSS_B,Win
Season,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
DayNum,NaN,1.000000,0.007189,0.015369,NaN,0.037815,0.041187,0.064772,0.008280,0.000221,0.043173,0.048840,-0.000818,0.010948,0.016335,0.021993,0.016155,0.032293,0.009906,0.020761,-0.010682,-0.008213
A,NaN,0.007189,1.000000,0.006501,NaN,0.046818,0.048674,0.059055,0.015141,0.039606,0.077238,0.043040,0.018254,-0.009517,0.036811,0.004694,0.004111,0.018520,0.043897,-0.007215,-0.000540,0.015884
B,NaN,0.015369,0.006501,1.000000,NaN,0.007950,0.024549,0.016182,-0.025694,0.032087,0.000265,0.033117,-0.007871,0.005360,0.013981,-0.016986,0.016887,0.024764,0.016943,-0.025379,-0.007490,-0.017235
Gender,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
PM_A,NaN,0.037815,0.046818,0.007950,NaN,1.000000,0.248968,0.542804,0.334653,0.153330,0.498720,0.491148,0.127556,0.030957,0.499231,0.291262,0.136801,0.482978,0.481202,0.120954,0.001059,0.234739
PM_B,NaN,0.041187,0.048674,0.024549,NaN,0.248968,1.000000,0.484332,0.295981,0.133869,0.481330,0.464432,0.116668,-0.014397,0.554546,0.361548,0.195135,0.537236,0.478271,0.129930,0.044212,-0.248389
margin_FGP2_A,NaN,0.064772,0.059055,0.016182,NaN,0.542804,0.484332,1.000000,0.208326,0.079433,0.485999,0.302932,0.110229,0.020530,0.316550,0.176212,0.023722,0.297939,0.463236,0.186514,0.001014,0.016242
margin_FGP3_A,NaN,0.008280,0.015141,-0.025694,NaN,0.334653,0.295981,0.208326,1.000000,0.178097,0.318931,0.116386,-0.035913,-0.005498,0.175619,0.056355,0.048305,0.118772,0.133006,0.103602,0.011820,0.035112
margin_FTP_A,NaN,0.000221,0.039606,0.032087,NaN,0.153330,0.133869,0.079433,0.178097,1.000000,0.073905,0.027070,-0.073855,-0.006092,0.001585,0.029612,-0.034075,0.090475,0.136835,-0.108075,0.020790,0.010482


In [23]:
final_data = get_data_reg()

4680it [00:12, 375.60it/s]00<?, ?it/s]
4635it [00:12, 382.31it/s]12<04:09, 12.49s/it]
4739it [00:13, 360.19it/s]24<03:53, 12.29s/it]
4821it [00:12, 383.76it/s]37<03:48, 12.69s/it]
5107it [00:13, 383.50it/s]50<03:35, 12.65s/it]
5227it [00:14, 368.55it/s]03<03:26, 12.90s/it]
5313it [00:14, 359.53it/s]17<03:20, 13.35s/it]
10427it [00:28, 370.86it/s]2<03:13, 13.82s/it]
10460it [00:27, 380.03it/s]0<03:59, 18.39s/it]
10433it [00:28, 363.25it/s]8<04:15, 21.26s/it]
10593it [00:28, 377.55it/s]57<04:19, 23.58s/it]
10718it [00:28, 382.19it/s]25<04:09, 24.97s/it]
10698it [00:28, 380.78it/s]53<03:53, 25.92s/it]
10708it [00:28, 380.26it/s]21<03:32, 26.59s/it]
10735it [00:29, 364.89it/s]49<03:09, 27.08s/it]
10744it [00:29, 367.31it/s]19<02:46, 27.81s/it]
10833it [00:28, 386.35it/s]48<02:21, 28.26s/it]
10499it [00:27, 386.69it/s]16<01:52, 28.21s/it]
7540it [00:19, 390.98it/s]:43<01:23, 27.91s/it]
10539it [00:27, 388.25it/s]03<00:50, 25.33s/it]
10976it [00:28, 388.12it/s]30<00:25, 25.89s/it]
100%|█████

In [24]:
final_data.to_csv('computed_data/reg_avg_data.csv',index=False)

In [25]:
data_03.to_csv('computed_data/data_03.csv',index=False)

In [26]:
final_data = pd.read_csv('computed_data/reg_avg_data.csv')

In [27]:
final_data.shape

(173017, 22)